La prueba es la siguiente:
    URL api: https://www.metaweather.com/api/

La prueba consiste en realizar un TestCase que utilice invocaciones a servicios REST con la herramienta SoapUI.

Se utilizan los 2 siguientes servicios (información en la URL indicada antes):

- Location Search - ejecutaremos con parámetro de URL query pasándole el nombre de la ciudad a consultar. Se obtiene el ID necesario para ejecutar un servicio que nos de la información metereológica ("woeid").

- Location - ejecutaremos con el identificador obtenido en el servicio anterior ("woeid") para consultar la información de la ciudad deseada. Se obtiene la información metereológica para los próximos 5 días.



In [1]:
# libreriá para las consultas
import requests

In [2]:
# función para extraer el id de la ciudad
def idCity(city):
    
    try:
        # llamada a la api
        r = requests.get('https://www.metaweather.com/api/location/search/', params={'query':city.lower()})

        if 'woeid' in r.text:
            # devuelve la id
            return r.json()[0]['woeid']
        else:
            # devuelve el mensaje de error
            return print(f'\nLo sentimos {city} no es un lugar registrado')
    
    except Exception as error:
        return print('\nSe ha producido un error al llamar al servicio: ', error)

- _En esta primera función pasamos como parámetro el nombre de la ciudad._ 
- _A continuación encontramos un bloque `try - excetp` para controlar los errores que pudiera generar la GETrequest a la api._ 
- _Dentro del bloque `try`, en la GETrequest pasamos la dirección de la api y el parámetro forzado en minúsculas. Seguidamente le preguntamos a la respuesta que nos devuelve si el `string` contiene la palabra `'woeid'`. Para el valor `True` devuelve la id que contiene, para el valor `False` quiere decir que no contiene la ciudad pedida y devuelve un mensaje de error._

In [3]:
idCity('Berlin')

638242

In [4]:
idCity('Gijon')


Lo sentimos Gijon no es un lugar registrado


In [5]:
# función para extraer la predicción de un día   
def location(city):
    # extraemos la id
    woeid = idCity(city)
    
    # si woeid es un número
    if type(woeid) == int: 
        try:
            # llamada a la api
            r = requests.get('https://www.metaweather.com/api/location/'+ str(woeid))
            return r.json()

        except Exception as error:
            return print('\nSe ha producido un error al llamar al servicio: ', error)
    
    # woeid no es número
    else:
        return woeid

- _A la función `location` le pasamos el parámetro con el nombre de la ciudad para llamar a la función anterior `idCity()`_

- _En el primer bloque `if - else` verificamos si la función ha devuelto un número. Para el caso `True` llamamos a la api concatenando la id, controlamos el posible error, y devolvemos la respuesta en forma de json . Para el caso `False` mostramos el mensaje de error que produce `idCity()`.

In [6]:
# ejemplo de llamada correcta 
location("Madrid")

{'consolidated_weather': [{'id': 4632576439353344,
   'weather_state_name': 'Clear',
   'weather_state_abbr': 'c',
   'wind_direction_compass': 'SW',
   'created': '2021-07-27T10:12:55.183842Z',
   'applicable_date': '2021-07-27',
   'min_temp': 18.835,
   'max_temp': 33.144999999999996,
   'the_temp': 29.985,
   'wind_speed': 5.247488821689335,
   'wind_direction': 218.52788702839086,
   'air_pressure': 1014.0,
   'humidity': 43,
   'visibility': 14.321052553089954,
   'predictability': 68},
  {'id': 5423285220147200,
   'weather_state_name': 'Clear',
   'weather_state_abbr': 'c',
   'wind_direction_compass': 'WSW',
   'created': '2021-07-27T10:12:58.254064Z',
   'applicable_date': '2021-07-28',
   'min_temp': 20.86,
   'max_temp': 35.11,
   'the_temp': 33.66,
   'wind_speed': 5.375084881794321,
   'wind_direction': 236.87700576133412,
   'air_pressure': 1011.5,
   'humidity': 30,
   'visibility': 14.58296051061799,
   'predictability': 68},
  {'id': 5214503336673280,
   'weather_stat

In [7]:
# Ciudad que no está presente en el registro
location("Gijon")


Lo sentimos Gijon no es un lugar registrado


In [8]:
# Si nos equivocamos al escribir
location("Madridddd")


Lo sentimos Madridddd no es un lugar registrado


---
Usando la estructura TestCase SoapUI realizar un desarrollo que:

- Dado un conjunto de ciudades definidas en una hoja de propiedades (step Properties), donde se puede tener de 1 a N propiedades (ejemplo: city1, city2, city3 ... cityN) y valores con el nombre de cada ciudad (ejemplo: London, Madrid, Barcelona, ... Caracas).
- Iterar cada ciudad para invocar al servicio 1 para extraer su ID correspondiente e invocar nuevamente al servicio 2 con el ID obtenido en la respuesta del primer servicio para obtener la información metereológica.
- El objetivo del test es pintar en consola al finalizar la ciudad y el día en el que se dará la mayor temperatura máxima.
---


In [9]:
import pandas as pd

# dataframe de capitales
df = pd.DataFrame({'Capitals': ['Dublin', 'Berlin', 'Riga', 'Oslo', 'Moskow', 'Vienna', 'Madrid', 'Paris']} )
df

,Capitals
0,Dublin
1,Berlin
2,Riga
3,Oslo
4,Moskow
5,Vienna
6,Madrid
7,Paris


In [10]:
# iteramos sobre el dataframe
for row in df.itertuples():
    
    # llamada a la función location
    pred = location(row[1])

    if type(pred) == dict:
        
        # diccionario de temperaturas
        temperatures = {item['applicable_date']:item['max_temp'] for item in pred['consolidated_weather']}
        
        maxtemp = temperatures[max(temperatures)]
        maxdate = max(temperatures)

        print(f'\nCIUDAD: {row[1]}\nTemperatura máxima: {round(maxtemp, 3)}, registrada el {maxdate}')
    else:
        continue


CIUDAD: Dublin
Temperatura máxima: 27.165, registrada el 2021-08-01

CIUDAD: Berlin
Temperatura máxima: 23.685, registrada el 2021-08-01

Lo sentimos Riga no es un lugar registrado

CIUDAD: Oslo
Temperatura máxima: 18.885, registrada el 2021-08-01

Lo sentimos Moskow no es un lugar registrado

CIUDAD: Vienna
Temperatura máxima: 21.95, registrada el 2021-08-01

CIUDAD: Madrid
Temperatura máxima: 29.82, registrada el 2021-08-01

CIUDAD: Paris
Temperatura máxima: 22.745, registrada el 2021-08-01


- He creado un dataframe en pandas para recoger los nombres de las capitales. Sobre ese dataframe iteramos para extraer el nombre de la capital y pasarlo a la función `location`.
- Como función devuelve un string o un diccionario. Si se trata de un diccionario acudimos a la lista que contiene `'consolidated_weather'` para recorrerla y crear un diccionario de fechas y tempreraturas. 
- Asignamos a `maxtemp` y a `maxdate` la temperatura y la fecha del día más caluroso del diccionario mediante el uso de la función incorporada `max()` y lo imprimimos por pantalla.
---
- Adicionalmente en la siguiente celda se ve como usar el dataframe para insertar la información tomada de la api.

In [11]:
# listas para recoger los datos
list_temps, list_dates = [],[]

# iteramos sobre el dataframe
for row in df.itertuples():
    
    # llamamos a la función
    pred = location(row[1])
    
    if type(pred) == dict:
        # diccionario de temperaturas
        temperatures = {item['applicable_date']:item['max_temp'] for item in pred['consolidated_weather']}
        maxtemp = temperatures[max(temperatures)]
        maxdate = max(temperatures)
        
        # añadimos los datos
        list_temps.append(maxtemp)
        list_dates.append(maxdate)

        print(f'\nCIUDAD: {row[1]}\nTemperatura máxima: {round(maxtemp, 3)}, registrada el {maxdate}')
    else:
        # añadimos los errores
        list_temps.append('error')
        list_dates.append('error')
        continue

# insertamos la información
df.insert(df.shape[1], 'temp_max', list_temps)
df.insert(df.shape[1], 'fecha_max', list_dates)
df


CIUDAD: Dublin
Temperatura máxima: 27.165, registrada el 2021-08-01

CIUDAD: Berlin
Temperatura máxima: 23.685, registrada el 2021-08-01

Lo sentimos Riga no es un lugar registrado

CIUDAD: Oslo
Temperatura máxima: 18.885, registrada el 2021-08-01

Lo sentimos Moskow no es un lugar registrado

CIUDAD: Vienna
Temperatura máxima: 21.95, registrada el 2021-08-01

CIUDAD: Madrid
Temperatura máxima: 29.82, registrada el 2021-08-01

CIUDAD: Paris
Temperatura máxima: 22.745, registrada el 2021-08-01


,Capitals,temp_max,fecha_max
0,Dublin,27.165,2021-08-01
1,Berlin,23.685,2021-08-01
2,Riga,error,error
3,Oslo,18.885,2021-08-01
4,Moskow,error,error
5,Vienna,21.950,2021-08-01
6,Madrid,29.820,2021-08-01
7,Paris,22.745,2021-08-01
